In [14]:
import importlib

import config
importlib.reload(config)
import utils
importlib.reload(utils)

import numpy as np
import os
import pandas as pd

import sys
sys.path.append(config.config["folder"]["code"])

from makeFactorPosition import *
from makeNetWorthFuture import MakeNetWorth

In [58]:
# config
cfg = config.config

bTrading = True
sDataPath = cfg["folder"]["cross"]
sFactorPath = os.path.join(sDataPath, "因子")
sRstPath = os.path.join(sDataPath, "result")

# 因子制作
dfFactor = -pd.read_pickle(os.path.join(sFactorPath, "roll_return.pkl")).iloc[:-1]
# dfFactor = pd.read_csv("data/position/position.csv", index_col=0, parse_dates=True)
# dfFactor = pd.read_csv("data/jm_comp.csv", index_col=0, parse_dates=True)
# dfFactor.columns = ["jm"]
# dfFactor.index = dfFactor.index.strftime("%Y%m%d").astype(int)
sFactorName = "测试"

weight = False  #  等权
# weight = dfFactor
nLongRatio = 1 / 3
# nLongRatio = dfFactor[dfFactor > 0]
nShortRatio = 1 / 3
# nShortRatio = dfFactor[dfFactor < 0]
nStartDate = 20150101
# nMinOffset = 1
nFee = 0.0002
# sTradingTime = 'close'
sTradingTime = "0950"
nFactorLag = 1

In [59]:
dfFactor

,A,AG,AL,AO,AP,AU,B,BB,BC,BR,...,TC,TS,UR,V,WH,WR,WT,Y,ZC,ZN
date,,,,,,,,,,,,,,,,,,,,,
20100104,0.027409,NaN,-0.023079,NaN,NaN,-0.001607,-0.009414,NaN,NaN,NaN,...,NaN,NaN,NaN,0.047003,-0.006005,-0.011594,-0.037466,-0.024378,NaN,-0.011201
20100105,0.030271,NaN,-0.017461,NaN,NaN,0.001743,0.012061,NaN,NaN,NaN,...,NaN,NaN,NaN,0.038553,-0.004064,-0.008321,NaN,-0.006926,NaN,-0.003706
20100106,0.028117,NaN,-0.005822,NaN,NaN,NaN,0.025101,NaN,NaN,NaN,...,NaN,NaN,NaN,0.014010,-0.003648,-0.008976,-0.025916,-0.023577,NaN,-0.000707
20100107,0.016676,NaN,-0.005288,NaN,NaN,-0.004287,-0.010423,NaN,NaN,NaN,...,NaN,NaN,NaN,0.014179,-0.003149,-0.039124,NaN,-0.008239,NaN,-0.003473
20100108,0.012443,NaN,-0.007928,NaN,NaN,-0.002646,0.009333,NaN,NaN,NaN,...,NaN,NaN,NaN,0.004614,-0.008046,-0.006532,-0.059168,-0.014223,NaN,-0.005684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20240418,0.081835,-0.000000,-0.000000,NaN,0.088124,-0.006814,-0.053706,NaN,-0.006963,NaN,...,NaN,NaN,0.036819,-0.092763,NaN,-0.147050,NaN,-0.008082,NaN,-0.000000
20240419,0.148534,-0.003442,-0.001133,NaN,0.092211,-0.008876,-0.121474,NaN,-0.012710,NaN,...,NaN,NaN,0.037186,-0.105016,NaN,-0.172462,NaN,-0.013816,NaN,-0.003677
20240422,0.186658,-0.002910,-0.000000,NaN,0.086318,-0.007337,-0.163557,NaN,-0.007552,NaN,...,NaN,NaN,0.036726,-0.095911,NaN,-0.264363,NaN,-0.009325,NaN,-0.006731


In [16]:
dfFactor = pd.read_csv("data/position/alpha_lbn_1_black.csv", index_col=0, parse_dates=True)
dfFactor.index = dfFactor.index.strftime("%Y%m%d")
dfFactor.index.name
dfFactor = utils.few2crossing(dfFactor)
dfFactor.to_csv("data/position/position.csv")

date
20121205    0
20121206    0
20121207    0
20121210    0
20121211    0
           ..
20240313    0
20240314    0
20240315    0
20240318    0
20240319    0
Name: FU, Length: 2741, dtype: int64

: 

In [60]:
!python backtest.py "data/position/position.csv"

Traceback (most recent call last):
  File "backtest.py", line 32, in <module>
    dfFactor = -pd.read_pickle(sFactorPath).iloc[:-1]
  File "c:\Users\ifwha\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\io\pickle.py", line 196, in read_pickle
    return pickle.load(handles.handle)
_pickle.UnpicklingError: could not find MARK


In [52]:
if bTrading:
    sRstPath = os.path.join(sRstPath + "-trading", sFactorName)
else:
    sRstPath = os.path.join(sRstPath, sFactorName)
if not os.path.exists(sRstPath):
    os.makedirs(sRstPath)

FactorPositionOutput(
    sRstPath,
    sFactorName,
    dfFactor,
    sDataPath,
    nStartDate,
    weight,
    nLongRatio,
    nShortRatio,
    nFactorLag=nFactorLag,
    bTrading=bTrading,
)
sRstFile = os.path.join(sRstPath, "%s.csv" % sFactorName)
print("读取持仓文件生成净值: %s" % sRstFile)
MakeNetWorth(
    sRstFile,
    bTrading=bTrading,
    nFee=nFee,
    sTradingTime=sTradingTime,
    bStatNetByCode=False,
).run()

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/ifwha/OneDrive - CUHK-Shenzhen/桌面/工作/A06_策略研究/黑色板块策略\\dominant.pkl'